# explorer

> CLI tool for API exploration of cjm-fasthtml-tailwind utilities

In [ ]:
#| default_exp cli.explorer

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Dict, List, Tuple, Any, Optional
from cjm_fasthtml_tailwind.cli.utils import (
    SearchResult, discover_utility_modules,
    load_code_from_file, display_items_generic, handle_module_not_found,
    find_usage_in_items, get_view_command, format_usage_examples, simple_item_formatter,
    indented_item_formatter, list_utility_modules
)
from cjm_fasthtml_tailwind.cli.factory_extraction import FactoryInfo, extract_factories_from_module, list_all_factories
from cjm_fasthtml_tailwind.cli.example_discovery import ExampleInfo, extract_test_examples_from_module, list_all_examples
from cjm_fasthtml_tailwind.cli.helper_discovery import get_module_helpers
from cjm_fasthtml_tailwind.cli.search import search_all
from cjm_fasthtml_tailwind.cli.core_utils_discovery import get_core_utilities
from cjm_fasthtml_tailwind.cli.test_code import execute_test_code

In [ ]:
#| export
def get_recommended_imports(
    modules: Optional[List[str]] = None  # Specific modules to include, or None for all
) -> List[str]:  # List of import statements
    """Get recommended import statements for using the library."""
    imports = []
    
    # Always include FastHTML
    imports.append("from fasthtml.common import *")
    
    # Get imports for utility modules
    for module_name, module in discover_utility_modules():
        if modules and module_name not in modules:
            continue
            
        # Get factories and helpers from this module
        factories = extract_factories_from_module(module, module_name)
        helpers = get_module_helpers(module_name)
        
        # Combine all names
        all_names = []
        if factories:
            all_names.extend([f.name for f in factories])
        if helpers:
            all_names.extend([h.name for h in helpers])
        
        if all_names:
            import_list = ", ".join(sorted(all_names))
            imports.append(f"from cjm_fasthtml_tailwind.utilities.{module_name} import {import_list}")
    
    # Add core utilities
    core_utils = get_core_utilities()
    for util in core_utils:
        imports.append(util.import_statement)
    
    return imports

## CLI Functions

Main functions for the CLI explorer tool:

In [ ]:
#| export
def list_module_factories(
    module_name: str  # Name of the module to inspect (e.g., 'spacing', 'sizing')
) -> List[FactoryInfo]:  # List of FactoryInfo objects for the module
    """List all factory instances in a specific utility module."""
    # Find the module
    for name, module in discover_utility_modules():
        if name == module_name:
            return extract_factories_from_module(module, module_name)
    
    return []  # Module not found

In [ ]:
#| export
def list_module_examples(
    module_name: str  # Name of the module to inspect
) -> List[ExampleInfo]:  # List of ExampleInfo objects
    """List all test example functions in a specific utility module."""
    # Find the module
    for name, module in discover_utility_modules():
        if name == module_name:
            return extract_test_examples_from_module(module, module_name)
    
    return []  # Module not found

In [ ]:
#| export
def get_example_by_name(
    module_name: str,  # Name of the module
    feature: str  # Feature name (e.g., 'basic', 'directional')
) -> Optional[ExampleInfo]:  # ExampleInfo object or None if not found
    """Get a specific example by module name and feature."""
    examples = list_module_examples(module_name)
    
    for example in examples:
        if example.feature == feature:
            return example
    
    return None

In [ ]:
#| export
def get_factory_by_name(
    module_name: str,  # Name of the module
    factory_name: str  # Name of the factory (e.g., 'p', 'w', 'flex')
) -> Optional[FactoryInfo]:  # FactoryInfo object or None if not found
    """Get a specific factory by module name and factory name."""
    factories = list_module_factories(module_name)
    
    for factory_info in factories:
        if factory_info.name == factory_name:
            return factory_info
    
    return None

## Display Functions

Functions to format and display the discovered information:

In [ ]:
#| export
def display_modules():
    """Display all available utility modules with their documentation."""
    modules = list_utility_modules()
    
    # Convert dict to list of tuples for the formatter
    module_list = [(name, doc) for name, doc in modules.items()]
    
    # Create formatter
    def module_formatter(item):
        name, doc = item
        return f"{name}: {doc}"
    
    # Instructions
    instructions = []
    if modules:
        first_module = list(modules.keys())[0]
        instructions.extend([
            ("To explore a specific module:", "cjm-tailwind-explore factories -m <module>"),
            ("Example:", f"cjm-tailwind-explore factories -m {first_module}")
        ])
    
    display_items_generic(
        items=module_list,
        title="Available Utility Modules:",
        item_formatter=module_formatter,
        item_type="modules",
        instructions=instructions
    )

In [ ]:
#| export
def display_module_factories(module_name: str):
    """Display all factories in a specific module."""
    factories = list_module_factories(module_name)
    
    if not factories:
        handle_module_not_found("factories", module_name)
        return
    
    print(f"Factories in '{module_name}' module:")
    print("=" * 60, "\n")

    for factory_info in factories:
        print(f"{factory_info.name}: {factory_info.doc}")
    
    print(f"\nTotal factories: {len(factories)}")
    
    # Add helpful instructions
    print("\nTo explore further:")
    print(f"  View detailed factory info:")
    if factories:
        first_factory = factories[0].name
        print(f"    cjm-tailwind-explore factory {module_name} {first_factory}")
    
    print(f"\n  View usage examples:")
    print(f"    cjm-tailwind-explore examples -m {module_name}")
    
    print(f"\n  View helper functions:")
    print(f"    cjm-tailwind-explore helpers -m {module_name}")
    
    print(f"\n  Search within this module:")
    print(f"    cjm-tailwind-explore search <query> --in factories")

In [ ]:
#| export
def display_all_factories():
    """Display all factories across all modules."""
    all_factories = list_all_factories()
    
    # Create formatter using simple_item_formatter
    factory_formatter = simple_item_formatter('name', 'doc')
    indented_formatter = indented_item_formatter("  ")
    
    # Instructions
    instructions = []
    if all_factories:
        first_module = list(all_factories.keys())[0]
        first_factory = all_factories[first_module][0].name if all_factories[first_module] else None
        
        instructions.extend([
            ("View factories for a specific module:", f"cjm-tailwind-explore factories -m {first_module}" if first_module else None),
            ("View detailed factory info:", f"cjm-tailwind-explore factory {first_module} {first_factory}" if first_module and first_factory else None),
            ("Search for specific factories:", "cjm-tailwind-explore search <query> --in factories")
        ])
    
    display_items_generic(
        items=all_factories,
        title="All Factories by Module:",
        item_formatter=indented_formatter(lambda item: f"{item.name} - {item.doc}"),
        item_type="factories",
        instructions=instructions
    )

In [ ]:
#| export
def display_module_examples(module_name: str):
    """Display all usage examples in a specific module."""
    examples = list_module_examples(module_name)
    
    if not examples:
        handle_module_not_found("examples", module_name)
        return
    
    print(f"Usage Examples in '{module_name}' module:")
    print("=" * 60)
    
    for example_info in examples:
        print(f"\n{example_info.feature.replace('_', ' ').title()} Examples:")
        print(f"Function: {example_info.name}")
        print("-" * 60)
        print(example_info.source)
    
    print(f"\nTotal example functions: {len(examples)}")
    
    # Add helpful instructions
    print("\nTo explore further:")
    print(f"  View a specific example:")
    if examples:
        first_example = examples[0]
        print(f"    cjm-tailwind-explore example {module_name} {first_example.feature}")
    
    print(f"\n  View factories in this module:")
    print(f"    cjm-tailwind-explore factories -m {module_name}")
    
    print(f"\n  Search for specific patterns in examples:")
    print(f"    cjm-tailwind-explore search <query> --in examples --include-source")

In [ ]:
#| export
def display_all_examples():
    """Display all usage examples across all modules."""
    all_examples = list_all_examples()
    
    # Create formatter
    def example_formatter(item):
        feature_name = item.feature.replace('_', ' ')
        return f"  - {feature_name}: {item.docstring}"
    
    display_items_generic(
        items=all_examples,
        title="All Usage Examples by Module:",
        item_formatter=example_formatter,
        item_type="example functions"
    )

In [ ]:
#| export
def display_example_source(module_name: str, feature: str):
    """Display the source code of a specific example function."""
    example = get_example_by_name(module_name, feature)
    
    if not example:
        print(f"No example found for feature '{feature}' in module '{module_name}'.")
        print(f"\nTry running 'cjm-tailwind-explore examples --module {module_name}' to see available features.")
        return
    
    print(f"Example: {example.feature.replace('_', ' ').title()} ({example.module_name} module)")
    print(f"Function: {example.name}")
    print("=" * 60)
    print(example.source)

In [ ]:
#| export
def display_module_helpers(module_name: str):
    """Display helper functions available in a specific module."""
    helpers = get_module_helpers(module_name)
    
    if not helpers:
        handle_module_not_found("helper functions", module_name)
        return
    
    print(f"Helper Functions in '{module_name}' module:")
    print("=" * 60)
    
    for helper_info in helpers:
        print(f"\n{helper_info.name}{helper_info.signature}")
        print("-" * 40)
        print(helper_info.docstring)
    
    print(f"\nTotal helper functions: {len(helpers)}")
    
    # Add helpful instructions
    print("\nTo explore further:")
    print(f"  View helper source code:")
    if helpers:
        first_helper = helpers[0].name
        print(f"    cjm-tailwind-explore helper {module_name} {first_helper}")
    
    print(f"\n  View factories in this module:")
    print(f"    cjm-tailwind-explore factories -m {module_name}")
    
    print(f"\n  View usage examples:")
    print(f"    cjm-tailwind-explore examples -m {module_name}")
    
    print(f"\n  Search for helper usage:")
    if helpers:
        print(f"    cjm-tailwind-explore search {helpers[0].name} --include-source")

In [ ]:
#| export
def display_helper_source(module_name: str, helper_name: str):
    """Display the source code of a specific helper function."""
    helpers = get_module_helpers(module_name)
    
    helper_info = None
    for h in helpers:
        if h.name == helper_name:
            helper_info = h
            break
    
    if not helper_info:
        print(f"No helper function '{helper_name}' found in module '{module_name}'.")
        print(f"\nTry running 'cjm-tailwind-explore helpers --module {module_name}' to see available helpers.")
        return
    
    print(f"Helper: {helper_info.name} ({helper_info.module_name} module)")
    print(f"Signature: {helper_info.name}{helper_info.signature}")
    print("=" * 60)
    print(helper_info.source)
    
    # Find test examples that demonstrate this helper
    examples = list_module_examples(module_name)
    if examples:
        # Look for the helper examples test function
        helper_example = None
        for example in examples:
            if example.feature == 'helper' or example.feature == 'helpers':
                helper_example = example
                break
        
        if helper_example:
            # Check if this specific helper is used in the test
            import re
            patterns = [
                rf'\b{helper_name}\(',           # helper_name(
                rf'\bstr\({helper_name}\b',      # str(helper_name
                rf'\({helper_name}\)',           # (helper_name)
            ]
            
            if any(re.search(pattern, helper_example.source) for pattern in patterns):
                print(f"\nTest Example:")
                print(f"  {helper_example.name} ({helper_example.docstring})")
                print(f"  View with: cjm-tailwind-explore example {module_name} {helper_example.feature}")
            else:
                print(f"\nNo test example found that demonstrates the '{helper_name}' helper.")
        else:
            print(f"\nNo helper examples test function found in module '{module_name}'.")

In [ ]:
#| export
def display_all_helpers():
    """Display all helper functions across all modules."""
    all_helpers = {}
    
    for module_name, module in discover_utility_modules():
        helpers = get_module_helpers(module_name)
        if helpers:
            all_helpers[module_name] = helpers
    
    # Create formatter
    def helper_formatter(item):
        # Get first line of docstring for concise display
        first_line = item.docstring.split('\n')[0] if item.docstring else "No documentation available"
        return f"  - {item.name}: {first_line}"
    
    display_items_generic(
        items=all_helpers,
        title="All Helper Functions by Module:",
        item_formatter=helper_formatter,
        item_type="helper functions"
    )

In [ ]:
#| export
def display_factory_info(module_name: str, factory_name: str):
    """Display detailed information about a specific factory."""
    factory_info = get_factory_by_name(module_name, factory_name)
    
    if not factory_info:
        print(f"No factory '{factory_name}' found in module '{module_name}'.")
        print(f"\nTry running 'cjm-tailwind-explore factories --module {module_name}' to see available factories.")
        return
    
    print(f"Factory: {factory_info.name} ({factory_info.module_name} module)")
    print("=" * 60)
    
    # Get detailed info from the factory
    try:
        info = factory_info.factory.get_info()
        
        # Display description
        print(f"\nDescription:")
        print(f"  {info['description']}")
        
        # Display valid inputs
        print(f"\nValid Inputs:")
        if isinstance(info['valid_inputs'], list):
            for input_desc in info['valid_inputs']:
                print(f"  - {input_desc}")
        else:
            print(f"  {info['valid_inputs']}")
        
        # Display options
        print(f"\nOptions:")
        if isinstance(info['options'], dict):
            for key, value in info['options'].items():
                if isinstance(value, dict):
                    print(f"  {key}:")
                    for sub_key, sub_value in value.items():
                        print(f"    - {sub_key}: {sub_value}")
                else:
                    print(f"  - {key}: {value}")
        else:
            print(f"  {info['options']}")
            
    except AttributeError:
        print("\nDetailed information not available for this factory.")
        print("Factory type:", type(factory_info.factory).__name__)
    
    # Find test examples that use this factory
    all_examples = list_all_examples()
    example_usage = find_usage_in_items(
        target_name=factory_name,
        items=all_examples,
        source_getter=lambda e: e.source,
        item_type="examples"
    )
    
    if example_usage:
        print(f"\nTest Examples Using This Factory:")
        formatted = format_usage_examples(
            usage_items=example_usage,
            item_name_getter=lambda e: e.name,
            item_type="examples",
            view_command_type="example"
        )
        for line in formatted:
            print(line)
    else:
        print(f"\nNo test examples found that specifically use the '{factory_name}' factory.")
    
    # Find helper functions that use this factory
    all_helpers = {}
    for mod_name, module in discover_utility_modules():
        helpers = get_module_helpers(mod_name)
        if helpers:
            all_helpers[mod_name] = helpers
    
    helper_usage = find_usage_in_items(
        target_name=factory_name,
        items=all_helpers,
        source_getter=lambda h: h.source,
        item_type="helpers"
    )
    
    if helper_usage:
        print(f"\nHelper Functions Using This Factory:")
        for mod_name, helper in helper_usage:
            # Get first line of docstring for concise display
            first_line = helper.docstring.split('\n')[0] if helper.docstring else "No description available"
            print(f"  - {helper.name}: {first_line}")
            print(f"    View with: {get_view_command('helper', mod_name, helper.name)}")
    else:
        print(f"\nNo helper functions found that specifically use the '{factory_name}' factory.")

In [ ]:
#| export
def display_search_results(results: List[SearchResult], query: str):
    """Display search results in a formatted way."""
    if not results:
        print(f"No results found for '{query}'")
        return
    
    print(f"Search Results for '{query}':")
    print("=" * 60)
    
    # Group results by content type
    by_type = {}
    for result in results:
        if result.content_type not in by_type:
            by_type[result.content_type] = []
        by_type[result.content_type].append(result)
    
    # Display results by type
    for content_type, type_results in by_type.items():
        print(f"\n{content_type.title()}s ({len(type_results)} results)")
        print("-" * 40)
        
        for result in type_results:
            print(f"\n  {result.item_name} ({result.module_name} module)")
            print(f"  Found in: {result.match_location}")
            print(f"  Context: {result.match_context}")
            
            # Show how to view the full item
            if result.content_type == 'factory':
                view_cmd = get_view_command('factory', result.module_name, result.item_name)
            elif result.content_type == 'example':
                # Extract feature from name
                feature = result.item_name.replace(f'test_{result.module_name}_', '').replace('_examples', '')
                view_cmd = get_view_command('example', result.module_name, feature)
            elif result.content_type == 'helper':
                view_cmd = get_view_command('helper', result.module_name, result.item_name)
            elif result.content_type == 'module':
                view_cmd = get_view_command('module', result.module_name, result.module_name)
            else:
                view_cmd = ""
            
            if view_cmd:
                print(f"  View with: {view_cmd}")
    
    print(f"\nTotal results: {len(results)}")

In [ ]:
#| export
def display_core_utility_source(util_name: str):
    """Display the source code of a specific core utility function."""
    utilities = get_core_utilities()
    
    util_info = None
    for u in utilities:
        if u.name == util_name:
            util_info = u
            break
    
    if not util_info:
        print(f"No core utility '{util_name}' found.")
        print(f"\nTry running 'cjm-tailwind-explore core-utils' to see available utilities.")
        return
    
    print(f"Core Utility: {util_info.name}")
    print(f"Module: {util_info.module}")
    print(f"Signature: {util_info.name}{util_info.signature}")
    print("=" * 60)
    print(util_info.source)
    
    # Find examples that use this utility
    print(f"\nFinding usage examples...")
    
    # Search in all test examples
    all_examples = list_all_examples()
    example_usage = find_usage_in_items(
        target_name=util_name,
        items=all_examples,
        source_getter=lambda e: e.source,
        item_type="examples"
    )
    
    if example_usage:
        print(f"\nFound in {len(example_usage)} test examples:")
        formatted = format_usage_examples(
            usage_items=example_usage,
            item_name_getter=lambda e: e.name,
            item_type="examples",
            view_command_type="example"
        )
        for line in formatted:
            print(line)
    
    # Search in helper functions
    all_helpers = {}
    for module_name, module in discover_utility_modules():
        helpers = get_module_helpers(module_name)
        if helpers:
            all_helpers[module_name] = helpers
    
    helper_usage = find_usage_in_items(
        target_name=util_name,
        items=all_helpers,
        source_getter=lambda h: h.source,
        item_type="helpers"
    )
    
    if helper_usage:
        print(f"\nFound in {len(helper_usage)} helper functions:")
        for module_name, helper in helper_usage:
            print(f"  - {helper.name} ({module_name} module)")
            print(f"    View with: {get_view_command('helper', module_name, helper.name)}")
    
    # Suggest search for more examples
    print(f"\nTo find more usage examples:")
    print(f"  cjm-tailwind-explore search {util_name} --include-source")

In [ ]:
#| export
def display_core_utilities():
    """Display all core utility functions."""
    utilities = get_core_utilities()
    
    if not utilities:
        print("No core utilities found.")
        return
    
    print("Core Utility Functions:")
    print("=" * 60)
    
    for util in utilities:
        print(f"\n{util.name}{util.signature}")
        print("-" * 40)
        print(f"Module: {util.module}")
        print(f"Import: {util.import_statement}")
        print(f"\nDocumentation:")
        print(util.docstring)
    
    print(f"\nTotal core utilities: {len(utilities)}")
    
    # Add helpful instructions
    print("\nTo explore further:")
    if utilities:
        first_util = utilities[0]
        print(f"  View source code:")
        print(f"    cjm-tailwind-explore core-util {first_util.name}")
    print(f"\n  Search for usage examples:")
    print(f"    cjm-tailwind-explore search combine_classes --include-source")

In [ ]:
#| export
def display_imports(modules: Optional[List[str]] = None):
    """Display recommended import statements."""
    imports = get_recommended_imports(modules)
    
    if modules:
        print(f"Recommended imports for modules: {', '.join(modules)}")
    else:
        print("Recommended imports for all modules:")
    print("=" * 60)
    print()
    
    for imp in imports:
        print(imp)
    
    print("\nUsage:")
    print("- Copy these imports to the top of your Python file")
    print("- The imports include all factories, helpers, and core utilities")
    print("- FastHTML components are available via 'from fasthtml.common import *'")
    
    if not modules:
        print("\nTo get imports for specific modules only:")
        print("  cjm-tailwind-explore imports --modules spacing sizing")

In [ ]:
#| export
def display_test_code_result(success: bool, stdout: str, stderr: str, code: str):
    """Display the results of test code execution."""
    print("Test Code Execution Result:")
    print("=" * 60)
    
    # Show the code that was tested
    print("\nCode tested:")
    print("-" * 40)
    print(code)
    print("-" * 40)
    
    # Show the result
    if success:
        print("\n✅ Code executed successfully!")
        if stdout:
            print("\nOutput:")
            print(stdout)
    else:
        print("\n❌ Code execution failed!")
        if stderr:
            print("\nError:")
            print(stderr)
    
    # Show helpful tips
    if not success:
        print("\nTips:")
        print("- Check that all factory names are correct (use 'cjm-tailwind-explore factories')")
        print("- Verify function signatures (use 'cjm-tailwind-explore helper <module> <name>')")
        print("- Look at examples for usage patterns (use 'cjm-tailwind-explore examples')")
        print("- Search for specific patterns (use 'cjm-tailwind-explore search <query>')")

## Parser Setup Functions

Helper functions to set up argument parsers for each command:

In [ ]:
#| export
def add_modules_parser(subparsers):
    """Add the 'modules' command parser."""
    return subparsers.add_parser('modules', help='List all utility modules')

In [ ]:
#| export
def add_factories_parser(subparsers):
    """Add the 'factories' command parser."""
    parser = subparsers.add_parser('factories', help='List factories')
    parser.add_argument(
        '--module', '-m',
        type=str,
        help='Show factories for a specific module only'
    )
    return parser

In [ ]:
#| export
def add_factory_parser(subparsers):
    """Add the 'factory' command parser."""
    parser = subparsers.add_parser('factory', help='Show detailed info for a specific factory')
    parser.add_argument(
        'module',
        type=str,
        help='Module name (e.g., spacing, sizing)'
    )
    parser.add_argument(
        'name',
        type=str,
        help='Factory name (e.g., p, m, w, h)'
    )
    return parser

In [ ]:
#| export
def add_examples_parser(subparsers):
    """Add the 'examples' command parser."""
    parser = subparsers.add_parser('examples', help='Show usage examples')
    parser.add_argument(
        '--module', '-m',
        type=str,
        help='Show examples for a specific module only'
    )
    return parser

In [ ]:
#| export
def add_example_parser(subparsers):
    """Add the 'example' command parser."""
    parser = subparsers.add_parser('example', help='Show source code for a specific example')
    parser.add_argument(
        'module',
        type=str,
        help='Module name (e.g., spacing, sizing)'
    )
    parser.add_argument(
        'feature',
        type=str,
        help='Feature name (e.g., basic, directional, negative)'
    )
    return parser

In [ ]:
#| export
def add_helpers_parser(subparsers):
    """Add the 'helpers' command parser."""
    parser = subparsers.add_parser('helpers', help='Show helper functions')
    parser.add_argument(
        '--module', '-m',
        type=str,
        help='Show helpers for a specific module only'
    )
    return parser

In [ ]:
#| export
def add_helper_parser(subparsers):
    """Add the 'helper' command parser."""
    parser = subparsers.add_parser('helper', help='Show source code for a specific helper')
    parser.add_argument(
        'module',
        type=str,
        help='Module name (e.g., spacing, sizing)'
    )
    parser.add_argument(
        'name',
        type=str,
        help='Helper function name (e.g., pad, margin)'
    )
    return parser

In [ ]:
#| export
def add_search_parser(subparsers):
    """Add the 'search' command parser."""
    parser = subparsers.add_parser('search', help='Search across all library components')
    parser.add_argument(
        'query',
        type=str,
        help='Search query'
    )
    parser.add_argument(
        '--in',
        dest='search_in',
        type=str,
        choices=['all', 'factories', 'examples', 'helpers', 'modules'],
        default='all',
        help='What to search in (default: all)'
    )
    parser.add_argument(
        '--include-source',
        action='store_true',
        help='Also search in source code of examples and helpers'
    )
    parser.add_argument(
        '--case-sensitive',
        action='store_true',
        help='Perform case-sensitive search'
    )
    return parser

In [ ]:
#| export
def add_test_code_parser(subparsers):
    """Add the 'test-code' command parser."""
    parser = subparsers.add_parser('test-code', help='Test code snippets using the library')
    parser.add_argument(
        'code',
        type=str,
        nargs='?',
        help='Code to test (or use --file to load from file)'
    )
    parser.add_argument(
        '--file', '-f',
        type=str,
        help='Load code from a file'
    )
    parser.add_argument(
        '--show-imports',
        action='store_true',
        help='Show the generated imports'
    )
    parser.add_argument(
        '--timeout',
        type=int,
        default=10,
        help='Execution timeout in seconds (default: 10)'
    )
    return parser

In [ ]:
#| export
def add_core_utils_parser(subparsers):
    """Add the 'core-utils' command parser."""
    return subparsers.add_parser('core-utils', help='List core utility functions')

In [ ]:
#| export
def add_core_util_parser(subparsers):
    """Add the 'core-util' command parser."""
    parser = subparsers.add_parser('core-util', help='Show source code for a core utility')
    parser.add_argument(
        'name',
        type=str,
        help='Utility name (e.g., combine_classes, get_tailwind_headers)'
    )
    return parser

In [ ]:
#| export
def add_imports_parser(subparsers):
    """Add the 'imports' command parser."""
    parser = subparsers.add_parser('imports', help='Show recommended import statements')
    parser.add_argument(
        '--modules', '-m',
        type=str,
        nargs='+',
        help='Specific modules to include (default: all modules)'
    )
    return parser

## Command Dispatcher

Function to handle command execution:

In [ ]:
#| export
def dispatch_command(args):
    """Dispatch the parsed arguments to the appropriate handler."""
    if args.command == 'modules':
        display_modules()
    
    elif args.command == 'factories':
        if args.module:
            display_module_factories(args.module)
        else:
            display_all_factories()
    
    elif args.command == 'factory':
        display_factory_info(args.module, args.name)
    
    elif args.command == 'examples':
        if args.module:
            display_module_examples(args.module)
        else:
            display_all_examples()
    
    elif args.command == 'example':
        display_example_source(args.module, args.feature)
    
    elif args.command == 'helpers':
        if args.module:
            display_module_helpers(args.module)
        else:
            display_all_helpers()
    
    elif args.command == 'helper':
        display_helper_source(args.module, args.name)
    
    elif args.command == 'search':
        handle_search_command(args)
    
    elif args.command == 'test-code':
        handle_test_code_command(args)
    
    elif args.command == 'core-utils':
        display_core_utilities()
    
    elif args.command == 'core-util':
        display_core_utility_source(args.name)
    
    elif args.command == 'imports':
        display_imports(args.modules)

In [ ]:
#| export
def handle_search_command(args):
    """Handle the search command."""
    # Determine which content types to search
    if args.search_in == 'all':
        content_types = None  # Will search all types
    else:
        content_types = [args.search_in]
    
    # Perform search
    results = search_all(
        args.query,
        content_types=content_types,
        include_source=args.include_source,
        case_sensitive=args.case_sensitive
    )
    
    # Display results
    display_search_results(results, args.query)

In [ ]:
#| export
def handle_test_code_command(args):
    """Handle the test-code command."""
    # Get code from command line or file
    code = None
    if args.file:
        code = load_code_from_file(args.file)
        if code is None:
            return  # Error already printed
    elif args.code:
        code = args.code
    else:
        print("Error: You must provide code to test either as an argument or via --file")
        print("\nExamples:")
        print('  cjm-tailwind-explore test-code "print(str(p(4)))"')
        print('  cjm-tailwind-explore test-code --file test_snippet.py')
        return
    
    # Execute the code
    success, stdout, stderr = execute_test_code(
        code,
        show_imports=args.show_imports,
        timeout=args.timeout
    )
    
    # Display results
    display_test_code_result(success, stdout, stderr, code)

## Main Parser Setup

Function to set up the main argument parser:

In [ ]:
#| export
def setup_argument_parser():
    """Set up the main argument parser with all subcommands."""
    parser = argparse.ArgumentParser(
        description="Explore cjm-fasthtml-tailwind utility factories and modules",
        prog="cjm-tailwind-explore"
    )
    
    # Create subcommands
    subparsers = parser.add_subparsers(dest='command', help='Available commands')
    
    # Add all subcommand parsers
    add_modules_parser(subparsers)
    add_factories_parser(subparsers)
    add_factory_parser(subparsers)
    add_examples_parser(subparsers)
    add_example_parser(subparsers)
    add_helpers_parser(subparsers)
    add_helper_parser(subparsers)
    add_search_parser(subparsers)
    add_test_code_parser(subparsers)
    add_core_utils_parser(subparsers)
    add_core_util_parser(subparsers)
    add_imports_parser(subparsers)
    
    return parser

## CLI Entry Point

Main function for the console script:

In [ ]:
#| export
import argparse

def main():
    """CLI entry point for exploring cjm-fasthtml-tailwind utilities."""
    # Set up the argument parser
    parser = setup_argument_parser()
    
    # Parse arguments
    args = parser.parse_args()
    
    # If no command specified, show help
    if args.command is None:
        parser.print_help()
        return
    
    # Dispatch to the appropriate command handler
    dispatch_command(args)

## CLI Usage

Once installed, you can use the CLI tool from the command line:

```bash
# Show help
cjm-tailwind-explore --help

# List all utility modules
cjm-tailwind-explore modules

# List all factories across all modules
cjm-tailwind-explore factories

# List factories for a specific module
cjm-tailwind-explore factories --module spacing
cjm-tailwind-explore factories -m sizing

# Show detailed info for a specific factory
cjm-tailwind-explore factory spacing p
cjm-tailwind-explore factory spacing m
cjm-tailwind-explore factory sizing w
cjm-tailwind-explore factory sizing h

# Show all usage examples across all modules
cjm-tailwind-explore examples

# Show usage examples for a specific module
cjm-tailwind-explore examples --module spacing
cjm-tailwind-explore examples -m sizing

# Show source code for a specific example
cjm-tailwind-explore example spacing basic
cjm-tailwind-explore example spacing directional
cjm-tailwind-explore example sizing width

# Show all helper functions across all modules
cjm-tailwind-explore helpers

# Show helper functions for a specific module
cjm-tailwind-explore helpers --module spacing
cjm-tailwind-explore helpers -m sizing

# Show source code for a specific helper function
cjm-tailwind-explore helper spacing pad
cjm-tailwind-explore helper spacing margin

# List core utility functions
cjm-tailwind-explore core-utils

# Show source code and usage for a core utility
cjm-tailwind-explore core-util combine_classes
cjm-tailwind-explore core-util get_tailwind_headers

# Show recommended import statements
cjm-tailwind-explore imports
cjm-tailwind-explore imports --modules spacing sizing
cjm-tailwind-explore imports -m flexbox_and_grid layout

# Search across all library components
cjm-tailwind-explore search padding
cjm-tailwind-explore search "margin"
cjm-tailwind-explore search negative

# Search only in specific content types
cjm-tailwind-explore search width --in factories
cjm-tailwind-explore search negative --in examples
cjm-tailwind-explore search pad --in helpers
cjm-tailwind-explore search spacing --in modules

# Search including source code
cjm-tailwind-explore search "p(" --include-source
cjm-tailwind-explore search "m.auto" --include-source
cjm-tailwind-explore search "combine_classes" --include-source

# Case-sensitive search
cjm-tailwind-explore search "BaseFactory" --case-sensitive

# Test code snippets
cjm-tailwind-explore test-code "print(str(p(4)))"
cjm-tailwind-explore test-code "print(str(w.full))"
cjm-tailwind-explore test-code "print(combine_classes(p(4), m.auto, 'flex'))"

# Test code from a file
cjm-tailwind-explore test-code --file test_snippet.py

# Test code with generated imports shown
cjm-tailwind-explore test-code "print(str(p(4)))" --show-imports

# Test code with custom timeout
cjm-tailwind-explore test-code --file complex_test.py --timeout 30
```

### Core Utilities

The `combine_classes` function is essential for combining utility classes:

```bash
# View documentation and source for combine_classes
cjm-tailwind-explore core-util combine_classes

# Search for usage examples
cjm-tailwind-explore search combine_classes --include-source

# Test it directly
cjm-tailwind-explore test-code "
result = combine_classes(
    p(4),
    m.auto,
    w.full,
    'bg-blue-500',
    None,  # None values are filtered out
    'text-white'
)
print('Combined:', result)
"
```

### Getting Started with Imports

```bash
# Get all recommended imports
cjm-tailwind-explore imports

# Get imports for specific modules only
cjm-tailwind-explore imports --modules spacing sizing

# Use the imports in a new file
cjm-tailwind-explore imports > my_imports.py
```

### Test Code Examples

The `test-code` command allows you to quickly test code snippets before implementing them:

```bash
# Test basic factory usage
cjm-tailwind-explore test-code "
# Test padding utilities
print('p(4):', str(p(4)))
print('p.x(8):', str(p.x(8)))
print('p.y.auto:', str(p.y.auto))
"

# Test combining classes
cjm-tailwind-explore test-code "
card_classes = combine_classes(
    p(6),
    m(4),
    'rounded-lg',
    'shadow-md'
)
print('Card classes:', card_classes)
"

# Test helper functions
cjm-tailwind-explore test-code "
# Using the pad helper
padding = pad(x=4, y=2)
print('Padding classes:', padding)

# Using the margin helper
spacing = margin(t=4, b=8, x='auto')
print('Margin classes:', spacing)
"

# Test a complete component
cjm-tailwind-explore test-code "
# Create a button component
button = Button(
    'Click me',
    cls=combine_classes(
        p.x(4),
        p.y(2),
        'bg-blue-500',
        'text-white',
        'rounded-md',
        'hover:bg-blue-600'
    )
)
print('Button HTML:', button)
"
```

## Export

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()